In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [2]:
out = 'out/20220728012207/'
df = pd.read_csv(out+'data.csv')
df['Stddev[%]'] = df['Stddev[%]'].str.rstrip('%').astype('float') / 100
z = 1.960
n = 100
df['CI@95%[ms]'] = z * df['Time[ms]'] * df['Stddev[%]'] / (n**0.5)
df

,Padding[bytes],Offset[bytes],Size[bytes],Time[ms],Stddev[%],CI@95%[ms]
0,0,8,16,16.9,0.30,0.993720
1,1,16,24,11.7,0.05,0.114660
2,2,16,24,10.8,0.21,0.444528
3,3,16,24,11.1,0.17,0.369852
4,4,16,24,11.4,0.05,0.111720
...,...,...,...,...,...,...
133,133,144,152,11.1,0.14,0.304584
134,134,144,152,10.3,0.33,0.666204
135,135,144,152,10.7,0.25,0.524300
136,136,144,152,11.3,0.10,0.221480


In [3]:
def line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands.
    Source: https://stackoverflow.com/a/69594497"""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [4]:
fig = line(
    data_frame = df,
    x = 'Padding[bytes]',
    y = 'Time[ms]',
    error_y = 'CI@95%[ms]',
    error_y_mode = 'band')
hl = 49
fig.add_trace(go.Scatter(
    x=[hl], 
    y=[df.at[hl,'Time[ms]']],
    mode='markers',
    marker=dict(color='red', size=6),
    showlegend=False))
fig.add_vline(
    x=hl,
    line_dash='dot',
    line_color='red',
    annotation=dict(text='Padding: 49B<br>   Offset: 64B',
                    font_color='red'))
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.write_image(out+'fig.png', scale=2)
fig.show()